In [17]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import time

# Scrape the first page in this topic

### Sorting out what I need

In [2]:
hexes_url = "https://www.mountainproject.com/forum/topic/118978980/recommendations-on-hexes"
uClient = uReq(hexes_url) # request the URL
page_html = uClient.read() # Read the html
uClient.close() # close the connection

In [3]:
hexes_soup = soup(page_html, "html.parser") # get soup of the webpage
hexes_soup.h1.text # topic name

'Recommendations on hexes'

In [4]:
message_container = hexes_soup.findAll("div", {"class":"fr-view"})

In [5]:
message_container[8] # no text in this one

<div class="fr-view">
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">

<?xml encoding='utf-8' ?><html><body><div><a href="https://cdn2.apstatic.com/forum/168709.jpg" rel="nofollow noopener noreferrer" target="_blank"><img class="img-fluid lazy forum-img" data-original="https://cdn2.apstatic.com/forum/168709.jpg"/></a></div></body></html>
</div>

In [6]:
message_container[0] # text of posting

<div class="fr-view">
<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">

<?xml encoding='utf-8' ?><html><body><p>Hello all, I'm trying to get into trad climbing but I'm also a starving college student. As I understand, hexes are a lightweight and cost-effective alternative to getting cams. Any recommendations on which ones are the best to get? Or should I just say screw it and purchase cams? Thanks!</p></body></html>
</div>

In [7]:
bio_container = hexes_soup.findAll("div", {"class":"bio text-truncate"})
bio_container[0].div.find_next('span').find_next('span').text.split("Joined ")[1].split("\n")[0] # date joined

'May 2020'

In [8]:
bio_container[0].div.find_next('a').find_next('a').strong.text # date posted

'Jun 3, 2020'

In [9]:
likes_container = hexes_soup.findAll("span", {"class":"num-likes text-muted"})
int(likes_container[0].text) # number of likes on the post

2

In [12]:
type(bio_container[13].div.find_next('a').find_next('a').strong.text)

str

In [13]:
mess_text = ""
for i in range(len(message_container[13].findAll('p'))):
    mess_text += message_container[13].findAll('p')[i].text

print(mess_text)

This is only a concern for your first piece of pro on each pitch and is easily mitigated:  place a cam for your first piece or use one of the bolts if its a bolted anchor or use the highest and/or best cam in your anchor.  I agree. Op spoke of using only nuts on a trad climb. I will add: “...highest and/or best cam in your anchor” on a multi pitch. 


### Putting it all together

In [14]:
# Get HTML info
hexes_url = "https://www.mountainproject.com/forum/topic/118978980/recommendations-on-hexes"
uClient = uReq(hexes_url) # request the URL
page_html = uClient.read() # Read the html
uClient.close() # close the connection

# Get soup and containers
hexes_soup = soup(page_html, "html.parser")
message_container = hexes_soup.findAll("div", {"class":"fr-view"})
bio_container = hexes_soup.findAll("div", {"class":"bio text-truncate"})
likes_container = hexes_soup.findAll("span", {"class":"num-likes text-muted"})

# Open a new file
filename = "hexes.csv"
f = open(filename, "w")
headers = "topic, text, join_date, post_date, num_likes\n"
f.write(headers)

# loop to get everything
for i in range(len(message_container)):
    topic = hexes_soup.h1.text
    try:
        mess_text = ""
        for j in range(len(message_container[i].findAll('p'))):
            mess_text += " " + message_container[i].findAll('p')[j].text
    except:
        mess_text = str(message_container[i].p) # outputs string "None" if image only
    join_date = bio_container[i].div.find_next('span').find_next('span').text.split("Joined ")[1].split("\n")[0]
    post_date = bio_container[i].div.find_next('a').find_next('a').strong.text
    num_likes = str(int(likes_container[i].text))
    
    f.write(topic +","+ 
            mess_text.replace(",", "~") +","+ 
            join_date +","+ 
            post_date.replace(",","") +","+ 
            num_likes + "\n")
    
f.close()
 

# Scraping all the pages in this topic

### Sorting out what I need

In [15]:
num_pages_container = hexes_soup.findAll("a", {"class":"no-click"})
num_pages = int(num_pages_container[2].text.split("of ")[1].split("\n")[0])
num_pages

5

In [16]:
# Open a new file
filename = "hexes.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)


# Get list of URLs from topic
hexes_url = "https://www.mountainproject.com/forum/topic/118978980/recommendations-on-hexes"
url_list = []
for i in range(1, num_pages+1):
    if i == 1:
        url = hexes_url
    else: 
        url = hexes_url + "?page=" + str(i)
    url_list.append(url)


for url in url_list:

    # Get HTML info
    hexes_url = url
    uClient = uReq(hexes_url) # request the URL
    page_html = uClient.read() # Read the html
    uClient.close() # close the connection

    # Get soup and containers
    hexes_soup = soup(page_html, "html.parser")
    message_container = hexes_soup.findAll("div", {"class":"fr-view"})
    bio_container = hexes_soup.findAll("div", {"class":"bio text-truncate"})
    likes_container = hexes_soup.findAll("span", {"class":"num-likes text-muted"})


    # loop to get everything
    for i in range(len(message_container)):
        topic = hexes_soup.h1.text
        page_num = url[-1]
        post_num = str(i)
        try:
            mess_text = ""
            for j in range(len(message_container[i].findAll('p'))):
                mess_text += " " + message_container[i].findAll('p')[j].text
        except:
            mess_text = str(message_container[i].p) # outputs empty string if image only
        join_date = bio_container[i].div.find_next('span').find_next('span').text.split("Joined ")[1].split("\n")[0]
        post_date = bio_container[i].div.find_next('a').find_next('a').strong.text
        num_likes = str(int(likes_container[i].text))

        f.write(topic +","+ 
                page_num +","+
                post_num +","+
                mess_text.replace(",", "~") +","+ 
                join_date +","+ 
                post_date.replace(",","") +","+ 
                num_likes + "\n")
    
f.close()

# Scraping all the topics

In [183]:
def forum_scrape(url_to_scrape):
    """Get all data from a given forum url"""
    
    # Get HTML info
    uClient = uReq(url_to_scrape) # request the URL
    page_html = uClient.read() # Read the html
    uClient.close() # close the connection
    
    # How many pages long is this forum topic?
    forum_soup = soup(page_html, "html.parser")
    num_pages_container = forum_soup.findAll("a", {"class":"no-click"})
    try:
        num_pages = int(num_pages_container[2].text.split("of ")[1].split("\n")[0])
    except:
        num_pages = 1

    # loop over each page
    url_list = []
    for page in range(1, num_pages+1):
        if page == 1:
            new_url = url_to_scrape
        else: 
            new_url = url_to_scrape + "?page=" + str(page)
        url_list.append(new_url)
    
    for url in url_list:
        time.sleep(1)
        
        # Get HTML info
        uClient = uReq(url) # request the URL
        page_html = uClient.read() # Read the html
        uClient.close() # close the connection

        # Get soup and containers
        forum_soup = soup(page_html, "html.parser")
        message_container = forum_soup.findAll("div", {"class":"fr-view"})
        bio_container = forum_soup.findAll("div", {"class":"bio text-truncate"})
        likes_container = forum_soup.findAll("span", {"class":"num-likes text-muted"})
    
        # get everything on the page
        for m in range(len(message_container)):
            
            topic = forum_soup.h1.text
            page_num = url[-1]
            post_num = str(m)
            try:
                mess_text = ""
                for j in range(len(message_container[m].findAll('p'))):
                    mess_text += " " + message_container[m].findAll('p')[j].text
            except:
                mess_text = str(message_container[m].p) # outputs empty string if image only
            
            join_date = bio_container[m].div.find_next('span').find_next('span').text.split("Joined ")[1].split("\n")[0]
            post_date = bio_container[m].div.find_next('a').find_next('a').strong.text
            if len(likes_container) == len(message_container):
                num_likes = str(int(likes_container[m].text))
            else:
                num_likes = str(10000)
            

            f.write(topic.replace(",", "~") +","+ 
                    page_num +","+
                    post_num +","+
                    mess_text.replace(",", "~") +","+ 
                    join_date +","+ 
                    post_date.replace(",","") +","+ 
                    num_likes + "\n")


In [ ]:
### Testing ###

# Open a new file
filename = "hexes.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)

# Test it out
url = "https://www.mountainproject.com/forum/topic/118116756/best-snag-free-very-light-trad-carabiner"
test_list = forum_scrape(url)

f.close()

### Get list of url of all topics
Then scrape that list

In [178]:
def get_topic_urls(main_discussion_url):
    """Get all subforum urls in a forum"""
    # Get HTML info
    uClient = uReq(main_discussion_url) # request the URL
    page_html = uClient.read() # Read the html
    uClient.close() # close the connection
    
    # How many pages of topics are there?
    forum_soup = soup(page_html, "html.parser")
    num_topic_pages_container = forum_soup.findAll("a", {"class":"no-click"})
    num_topic_pages = int(num_topic_pages_container[2].text.split("of ")[1].split("\n")[0])
    print("Scraping %s pages of forum URLs." %num_topic_pages)
    
    url_list = []
    for page in range(1, num_topic_pages+1):
        if page == 1:
            new_url = main_discussion_url
        else: 
            new_url = main_discussion_url + "?page=" + str(page)
        url_list.append(new_url)
    
    forum_list = []
    for url in url_list:
        time.sleep(.25)
        # Get HTML info
        uClient = uReq(url) # request the URL
        page_html = uClient.read() # Read the html
        uClient.close() # close the connection
        
        forum_soup = soup(page_html, "html.parser")
        forum_container = forum_soup.findAll("table", {"class":"table width100 table-striped"})
        for a in forum_soup.find_all('a', href=True):
            if "topic/" in a["href"]:
                if "forum-topic" not in a["href"]:
                    forum_list.append(a["href"])
                    
    print("There are %s forum URLs" %len(forum_list))   
    
    return forum_list

In [128]:
t0 = time.time()

main_url_discussion = "https://www.mountainproject.com/forum/103989417/climbing-gear-discussion"
all_discussion_urls = get_topic_urls(main_url_discussion)

main_url_reviews = "https://www.mountainproject.com/forum/106123795/climbing-gear-reviews"
all_review_urls = get_topic_urls(main_url_reviews)

t1 = time.time()
t1-t0

Scraping 261 pages of forum URLs.
There are 10414 forum URLs
Scraping 19 pages of forum URLs.
There are 739 forum URLs


183.49131202697754

### Use list of URLs to scrape each forum

In [174]:
t0 = time.time()

# Open a new file
filename = "reviews.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)

# Iterate over urls
for url in all_review_urls:
    forum_scrape(url)

f.close()

t1 = time.time()
t1-t0

865.2236521244049

In [ ]:
import requests
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'})
response.status_code
t0 = time.time()

# Open a new file
filename = "discussion1.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)

# Iterate over urls
for url in all_discussion_urls:
    forum_scrape(url)

f.close()

t1 = time.time()
t1-t0

In [193]:
f.close()

## The cell above got me 95722 rows

I need to extract some more.

The last forum scraped was "GEAR4ROCKS is no more? I should have bought their plastic nuts when I had the chance!" December 7, 2016 page 119.

Next forum is "Mammut Ropes" December 6,2017

In [190]:
# Url of Mammut Ropes forum
ropes_url = "https://www.mountainproject.com/forum/topic/112350107/mammut-ropes"

# Find location of this forum in list of all urls
ropes_index = all_discussion_urls.index(ropes_url)

second_part_discussion_urls = all_discussion_urls[ropes_index:]
len(second_part_discussion_urls) # 5677 urls left to scrape

5677

In [ ]:
import requests
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'})
response.status_code
t0 = time.time()

# Open a new file
filename = "discussion2.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)

# Iterate over urls
for url in second_part_discussion_urls:
    forum_scrape(url)

f.close()

t1 = time.time()
t1-t0

## The cell above got me 27199 rows

I need to extract the rest.

The last forum scraped was "EP Best Customer Service Ever" January 8 2015, page 162.

Next forum is "Advice on stoves" January 8,2015

The code below will extract the rest. Run when you are ready. 

In [198]:
# Url of Advice on stoves forum
stoves_url = "https://www.mountainproject.com/forum/topic/109900178/advice-on-stoves"

# Find location of this forum in list of all urls
stoves_index = all_discussion_urls.index(stoves_url)

third_part_discussion_urls = all_discussion_urls[stoves_index:]
len(third_part_discussion_urls) # 3963 urls left to scrape

3963

In [199]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [200]:
import requests
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36'})
response.status_code
t0 = time.time()

# Open a new file
filename = "discussion3.csv"
f = open(filename, "w")
headers = "topic, page_num, post_num, text, join_date, post_date, num_likes\n"
f.write(headers)

# Iterate over urls
for url in tqdm(third_part_discussion_urls):
    forum_scrape(url)

f.close()

t1 = time.time()
t1-t0

/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


9906.035131931305

In [196]:
f.close()

# Cleanup discussion forum

In [272]:
import glob
import pandas as pd
import numpy as np

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
disc_filenames = [i for i in all_filenames if "discussion" in i]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in disc_filenames ])

#export to csv
combined_csv.to_csv("discussion_forum.csv", index=False, encoding='utf-8-sig')

In [246]:
df = pd.read_csv("discussion_forum.csv")
df = df.rename(columns=lambda x: x.strip()) # remove whitespace from col names
df.head(30)

,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Katabatic quilts and down shift,t,0,Have a hammock gear quilt right now. I reall...,Jan 2012,1 day ago,1
1,Katabatic quilts and down shift,t,1,Do you roll or stuff the quilt when not in us...,Mar 2020,1 day ago,0
2,Katabatic quilts and down shift,t,2,I stuff it and every night the down ends up a...,Jan 2012,4 hours ago,0
3,Katabatic quilts and down shift,t,3,I've had one of the Katabatic 20 degree quilt...,Jun 2016,4 hours ago,0
4,Adidas owning 5.10,0,0,I know this is kind of old news~ but what is ...,Jun 2017,Nov 21 2019,0
5,Adidas owning 5.10,0,1,Adidas has owned 5.10 since 2011~ longer than...,Aug 2019,Nov 21 2019,22
6,Adidas owning 5.10,0,2,I think overwhelmingly~ the change has hurt t...,Jul 2017,Nov 21 2019,7
7,Adidas owning 5.10,0,3,Meh. That may have been when there was a tran...,Jan 2017,Nov 21 2019,12
8,Adidas owning 5.10,0,4,I dunno about the rubber~ but I have some moc...,Nov 2018,Nov 21 2019,0
9,Adidas owning 5.10,0,5,I wore guide tennis before and after Adidas t...,Oct 2008,Nov 21 2019,7


In [250]:
# Correct the page_num column

for i in range(len(df["page_num"])):
    try:
        df["page_num"][i] = int(df["page_num"][i])
    except:
        df["page_num"][i] = 1
    if df["page_num"][i] == 0:
        df["page_num"][i] = 1

df.head(20)

/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Katabatic quilts and down shift,1,0,Have a hammock gear quilt right now. I reall...,Jan 2012,1 day ago,1
1,Katabatic quilts and down shift,1,1,Do you roll or stuff the quilt when not in us...,Mar 2020,1 day ago,0
2,Katabatic quilts and down shift,1,2,I stuff it and every night the down ends up a...,Jan 2012,4 hours ago,0
3,Katabatic quilts and down shift,1,3,I've had one of the Katabatic 20 degree quilt...,Jun 2016,4 hours ago,0
4,Adidas owning 5.10,1,0,I know this is kind of old news~ but what is ...,Jun 2017,Nov 21 2019,0
5,Adidas owning 5.10,1,1,Adidas has owned 5.10 since 2011~ longer than...,Aug 2019,Nov 21 2019,22
6,Adidas owning 5.10,1,2,I think overwhelmingly~ the change has hurt t...,Jul 2017,Nov 21 2019,7
7,Adidas owning 5.10,1,3,Meh. That may have been when there was a tran...,Jan 2017,Nov 21 2019,12
8,Adidas owning 5.10,1,4,I dunno about the rubber~ but I have some moc...,Nov 2018,Nov 21 2019,0
9,Adidas owning 5.10,1,5,I wore guide tennis before and after Adidas t...,Oct 2008,Nov 21 2019,7


In [264]:
# Correct the post date column
date_change_needed = [date for date in df.post_date.unique() if "ago" in date]

date_dict = {7:"Jun 29 2020",
            6:"Jun 30 2020",
            5:"Jul 1 2020",
            4:"Jul 2 2020",
            3:"Jul 3 2020",
            2:"Jul 4 2020",
            1: "Jul 5 2020"}

replacement_dates = dict()
for date in date_change_needed:
    if "hour" in date:
        replacement_dates[date] = "Jul 6 2020"
        continue
    if "min" in date:
        replacement_dates[date] = "Jul 6 2020"
        continue
    if "7" in date:
        replacement_dates[date] = date_dict[7]
    if "6" in date:
        replacement_dates[date] = date_dict[6]
    if "5" in date:
        replacement_dates[date] = date_dict[5]
    if "4" in date:
        replacement_dates[date] = date_dict[4]
    if "3" in date:
        replacement_dates[date] = date_dict[3]
    if "2" in date:
        replacement_dates[date] = date_dict[2]
    if "1" in date:
        replacement_dates[date] = date_dict[1]

for i in range(len(df["post_date"])):
    try:
        df["post_date"][i] = replacement_dates[df["post_date"][i]]
    except:
        continue

df.head()

/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Katabatic quilts and down shift,1,0,Have a hammock gear quilt right now. I reall...,Jan 2012,Jul 5 2020,1
1,Katabatic quilts and down shift,1,1,Do you roll or stuff the quilt when not in us...,Mar 2020,Jul 5 2020,0
2,Katabatic quilts and down shift,1,2,I stuff it and every night the down ends up a...,Jan 2012,Jul 6 2020,0
3,Katabatic quilts and down shift,1,3,I've had one of the Katabatic 20 degree quilt...,Jun 2016,Jul 6 2020,0
4,Adidas owning 5.10,1,0,I know this is kind of old news~ but what is ...,Jun 2017,Nov 21 2019,0


In [275]:
# clean up join_date column
date_change_needed = []
for date in df.join_date.unique():
    if "20" not in date:
        if "19" not in date:
            date_change_needed.append(date)
            
for i in range(len(df["join_date"])):
    if df["join_date"][i] == "unknown":
        df["join_date"][i] == np.nan
        continue
    if df["join_date"][i] in date_change_needed:
        df["join_date"][i] = "Jun 2020"
df.join_date.unique()

array(['Jan 2012', 'Mar 2020', 'Jun 2016', 'Jun 2017', 'Aug 2019',
       'Jul 2017', 'Jan 2017', 'Nov 2018', 'Oct 2008', 'Sep 2019',
       'Mar 2016', 'Oct 2019', 'Apr 2007', 'Oct 2016', 'Jul 2014',
       'Jul 2019', 'Apr 2012', 'Sep 2013', 'Jan 2019', 'May 2015',
       'Jul 2015', 'Feb 2012', 'Feb 2014', 'May 2017', 'Feb 2013',
       'Jul 2018', 'Aug 2009', 'Apr 2010', 'May 2020', 'Aug 2008',
       'Mar 2014', 'Jan 2013', 'Jan 2016', 'Sep 2010', 'Oct 2013',
       'Aug 2016', 'Jul 2010', 'Sep 2018', 'Feb 2008', 'Nov 2015',
       'Mar 2009', 'Feb 2015', 'Jun 2019', 'Jan 2001', 'Feb 2006',
       'Mar 2006', 'Jun 2020', 'Nov 2017', 'Mar 2019', 'Aug 2018',
       'Dec 2013', 'Dec 2009', 'Nov 2009', 'Apr 2019', 'Mar 2018',
       'Aug 2015', 'Jun 2013', 'Apr 2017', 'Feb 2016', 'Nov 2006',
       'Oct 2015', 'Dec 2008', 'Dec 2015', 'Jun 2015', 'May 2016',
       'Sep 2017', 'Oct 2007', 'May 2014', 'Jun 2018', 'May 2018',
       'Apr 2018', 'Jun 2014', 'Apr 2013', 'Oct 2011', 'Jan 20

In [276]:
import os
os.getcwd()

'/Users/patriciadegner/Documents/MIDS/DL/final_project'

In [277]:
df.to_csv("/Users/patriciadegner/Documents/MIDS/DL/final_project/discussion_forum.csv", index = False, header = True)

# Cleanup review forum

In [278]:
df = pd.read_csv("review_forum.csv")
df = df.rename(columns=lambda x: x.strip()) # remove whitespace from col names
df.head(30)

,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Plaquette/Guide Style Device,e,0,Hi all~ I am in the market for a new plaquett...,Sep 2017,Jun 19 2020,0
1,Plaquette/Guide Style Device,e,1,Easiest on the elbows would definitely be a G...,May 2006,Jun 19 2020,1
2,Plaquette/Guide Style Device,e,2,DMM Pivot,Mar 2019,Jun 19 2020,4
3,Plaquette/Guide Style Device,e,3,Both the Pivot and the ATC guide accept ropes...,Jun 2015,Jun 19 2020,0
4,Plaquette/Guide Style Device,e,4,Thanks Adam! As a clarification~ When climbin...,Sep 2017,Jun 19 2020,0
5,Plaquette/Guide Style Device,e,5,Right on~ Chris. In that case~ the Pivot is ...,Jun 2015,Jun 19 2020,1
6,Plaquette/Guide Style Device,e,6,grigri,Sep 2010,Jun 19 2020,0
7,Plaquette/Guide Style Device,e,7,To be fair~ a Gigi adds very little weight or...,Aug 2011,Jun 19 2020,0
8,Plaquette/Guide Style Device,e,8,I cannot speak from direct experience~ but I ...,Aug 2013,Jun 19 2020,0
9,Plaquette/Guide Style Device,e,9,Hey Karl~ Thanks for your reply. I have tried...,Sep 2017,Jun 19 2020,0


In [279]:
# Correct the page_num column

for i in range(len(df["page_num"])):
    try:
        df["page_num"][i] = int(df["page_num"][i])
    except:
        df["page_num"][i] = 1
    if df["page_num"][i] == 0:
        df["page_num"][i] = 1

df.head(20)

/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__m

,topic,page_num,post_num,text,join_date,post_date,num_likes
0,Plaquette/Guide Style Device,1,0,Hi all~ I am in the market for a new plaquett...,Sep 2017,Jun 19 2020,0
1,Plaquette/Guide Style Device,1,1,Easiest on the elbows would definitely be a G...,May 2006,Jun 19 2020,1
2,Plaquette/Guide Style Device,1,2,DMM Pivot,Mar 2019,Jun 19 2020,4
3,Plaquette/Guide Style Device,1,3,Both the Pivot and the ATC guide accept ropes...,Jun 2015,Jun 19 2020,0
4,Plaquette/Guide Style Device,1,4,Thanks Adam! As a clarification~ When climbin...,Sep 2017,Jun 19 2020,0
5,Plaquette/Guide Style Device,1,5,Right on~ Chris. In that case~ the Pivot is ...,Jun 2015,Jun 19 2020,1
6,Plaquette/Guide Style Device,1,6,grigri,Sep 2010,Jun 19 2020,0
7,Plaquette/Guide Style Device,1,7,To be fair~ a Gigi adds very little weight or...,Aug 2011,Jun 19 2020,0
8,Plaquette/Guide Style Device,1,8,I cannot speak from direct experience~ but I ...,Aug 2013,Jun 19 2020,0
9,Plaquette/Guide Style Device,1,9,Hey Karl~ Thanks for your reply. I have tried...,Sep 2017,Jun 19 2020,0


In [281]:
# Correct the post date column
date_change_needed = [date for date in df.post_date.unique() if "ago" in date]

date_dict = {7:"Jun 29 2020",
            6:"Jun 30 2020",
            5:"Jul 1 2020",
            4:"Jul 2 2020",
            3:"Jul 3 2020",
            2:"Jul 4 2020",
            1: "Jul 5 2020"}

replacement_dates = dict()
for date in date_change_needed:
    if "hour" in date:
        replacement_dates[date] = "Jul 6 2020"
        continue
    if "min" in date:
        replacement_dates[date] = "Jul 6 2020"
        continue
    if "7" in date:
        replacement_dates[date] = date_dict[7]
    if "6" in date:
        replacement_dates[date] = date_dict[6]
    if "5" in date:
        replacement_dates[date] = date_dict[5]
    if "4" in date:
        replacement_dates[date] = date_dict[4]
    if "3" in date:
        replacement_dates[date] = date_dict[3]
    if "2" in date:
        replacement_dates[date] = date_dict[2]
    if "1" in date:
        replacement_dates[date] = date_dict[1]

for i in range(len(df["post_date"])):
    try:
        df["post_date"][i] = replacement_dates[df["post_date"][i]]
    except:
        continue

for date in df.post_date.unique():
    if "19" not in date:
        if "20" not in date:
            print(date)

In [282]:
# clean up join_date column
date_change_needed = []
for date in df.join_date.unique():
    if "20" not in date:
        if "19" not in date:
            date_change_needed.append(date)
            
for i in range(len(df["join_date"])):
    if df["join_date"][i] == "unknown":
        df["join_date"][i] == np.nan
        continue
    if df["join_date"][i] in date_change_needed:
        df["join_date"][i] = "Jun 2020"
df.join_date.unique()

/Users/patriciadegner/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


array(['Sep 2017', 'May 2006', 'Mar 2019', 'Jun 2015', 'Sep 2010',
       'Aug 2011', 'Aug 2013', 'Sep 2016', 'Aug 2008', 'Feb 2020',
       'Nov 2015', 'Apr 2015', 'Jan 2013', 'Jun 2013', 'May 2017',
       'Jun 2020', 'Apr 2020', 'Dec 2015', 'Jun 2016', 'Jul 2013',
       'Dec 2016', 'Nov 2009', 'Oct 2008', 'Sep 2013', 'Oct 2014',
       'Aug 2018', 'Jun 2019', 'May 2014', 'Mar 2012', 'Sep 2018',
       'Oct 2010', 'Nov 2014', 'Mar 2017', 'Oct 2013', 'Apr 2019',
       'Jun 2011', 'Oct 2017', 'Jun 2018', 'Feb 2018', 'Nov 2017',
       'May 2020', 'Oct 2015', 'Feb 2012', 'Feb 2010', 'May 2018',
       'Mar 2016', 'Apr 2011', 'Mar 2009', 'Mar 2013', 'Aug 2019',
       'Feb 2009', 'Aug 2015', 'Feb 2014', 'Feb 2017', 'Oct 2019',
       'Jun 2017', 'Mar 2018', 'Sep 2012', 'Jul 2018', 'Aug 2017',
       'Aug 2006', 'Feb 2006', 'Jan 2011', 'Jul 2015', 'Jun 2014',
       'Jul 2010', 'Oct 2018', 'Mar 2014', 'Feb 2015', 'May 2016',
       'Feb 2016', 'Sep 2015', 'May 2011', 'Jan 2016', 'Apr 20

In [283]:
df.to_csv("/Users/patriciadegner/Documents/MIDS/DL/final_project/review_forum.csv", index = False, header = True)